In [ ]:
# Indicadores que queremos encontrar
# VPA - Valor Patrimonial por Ação - 1.5x
# LPA - Lucro por Ação - 15x
# (22.5 * VPA * LPA) ** (1/2)

In [ ]:
# mocking - requests em massa
# zipfiles - biblioteca arquivo zip
# scrapy - biblioteca de scrapping
# https://dados.cvm.gov.br/dataset/cia_aberta-doc-itr - site

In [42]:
import fundamentus

list_tickers = fundamentus.get_resultado()
list_tickers = list(list_tickers.index)

c:\Users\Andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\fundamentus\resultado.py:50: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(content.text, decimal=",", thousands='.')[0]


In [67]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint
from fake_useragent import UserAgent

def extrair_indicadores(cell):
    indicador_data = {}
    
    indicador_data['nome'] = cell.find('span', class_='d-flex').text.strip("\n")
    
    valor_span = cell.find('div', class_='value').find('span')
    if valor_span:
        indicador_data['valor'] = valor_span.text.strip()
    
    return indicador_data

def get_data(ticker):
    url_info = f"https://investidor10.com.br/acoes/{ticker.lower()}/"
    
    headers = {"User-Agent": UserAgent().random}

    response = requests.get(url_info, headers=headers)
    if response.status_code != 200:
        return None
    
    soup = BeautifulSoup(response.content, "html.parser")
    table_indicators = soup.find("div", id="table-indicators")

    data = {}

    for cell in table_indicators.find_all("div", class_="cell"):
        indicador_data = extrair_indicadores(cell)
        data[indicador_data['nome']] = indicador_data['valor']

    print(f"Dados: {ticker}")
    return data


In [68]:
info = {}
for ticker in list_tickers[:10]:
    data = get_data(ticker)
    if data:
        conta = (22.5 * float(data["VPA "].replace(",", ".")) * float(data["LPA "].replace(",", "."))) ** (1/2)
        try:
            info[ticker] = round(conta, 2)
        except TypeError:
            info[ticker] = conta

Dados: AALR3
Dados: ABCB4
Dados: ABEV3
Dados: ADHM3


In [69]:
info

{'AALR3': (1.2676292338605709e-15+20.70195642928465j),
 'ABCB4': 44.58,
 'ABEV3': 11.13,
 'ADHM3': 3.22}